# Classification

given training set $ D=\left \{ \left ( x_{n},y_{n} \right )_{N}^{n=1} \right \} , y_{i}\in \left \{ 0,1 \right \} \forall i=1...N. $ Please note that the labels are not +1 or -1 as we discussed in lectures, so think carefully before you apply formula to this question. Your task is to learn a model $ w^{T}x + b $ that minimize the given loss. Please find the gradients of the above two loss functions by yourself and apply **average** gradient descent to update $w, b$ in each iteration. For perceptron loss we define $Z = y_{n}\left ( w^{T}x_{n}+b \right ) > 0$ as correctly classified data.

* (8 points) ```TODO 1```
For perceptron loss that is find the minimizer of $$ F\left ( w,b \right ) = \sum_{n=1}^{N}L_{perceptron}\left ( y_{n}\left ( w^{T}x_{n}+b \right ) \right ) = \sum_{n=1}^{N}MAX\left ( 0, -y_{n}\left ( w^{T}x_{n}+b \right )\right ) $$

* (8 points) ```TODO 2```
For logistic loss that is find the minimizer of $$ F\left ( w,b \right ) = \sum_{n=1}^{N}L_{logistic}\left ( y_{n}\left ( w^{T}x_{n}+b \right ) \right ) = \sum_{n=1}^{N}ln\left ( 1+e^{ -y_{n}\left ( w^{T}x_{n}+b \right )}\right ) $$

* (4 points) ```TODO 3```
Also you will find out it is convenient to use sigmoid fuction $\sigma \left (z\right)=(1+e^{-z})^{-1}$ for logistic loss, so please complete it. You can use this function in ```TODO 2```.

* (4 points for each)```TODO 4``` ```TODO 5```
After you learn the models, how do you know it is good enough? The intuitive way is to make some predictions to see if those predicted results are correct or not. Here we want you complete the prediction functions. It will be like something greater than 0 and something put into sigmoid function greater than 0.5. You may find out an interesting fact here.

After you finish the 5 ```TODO```s above in ```bm_classify.py```, you can run ```binary.sh``` to test it. If your code is programmed correctly, you should see ```binary.out``` as an output file keeping taining and testing accurancies of two loss functions for three datasets. You will see how similar between perceptron loss and logistic loss. Quite interesting isn't it? Can you figure out why this happends? We will leave this open question to you.

Two of the datasets you are going to do binary classification:
* Synthetic data: <img src="../Synthetic_data.png" width="400">
* Two Moon data: <img src="../Two_Moon_data.png" width="400">

In [1]:
from __future__ import division, print_function
import numpy as np
import bm_classify as sol

def accuracy_score(true, preds):
    return np.sum(true == preds).astype(float) / len(true)

from data_loader import toy_data_binary, \
                        moon_dataset, \
                        data_loader_mnist 

datasets = [(toy_data_binary(), 'Synthetic data'), 
            (moon_dataset(), 'Two Moon data'),
            (data_loader_mnist(), 'Binarized MNIST data')]

for data, name in datasets:
    print(name)
    X_train, X_test, y_train, y_test = data

    if name == 'Binarized MNIST data':
        y_train = [0 if yi < 5 else 1 for yi in y_train]
        y_test = [0 if yi < 5 else 1 for yi in y_test]
        y_train = np.asarray(y_train)
        y_test = np.asarray(y_test)

    for loss_type in ["perceptron", "logistic"]:
        w, b = sol.binary_train(X_train, y_train, loss=loss_type)
        train_preds = sol.binary_predict(X_train, w, b, loss=loss_type)
        preds = sol.binary_predict(X_test, w, b, loss=loss_type)
        print(loss_type + ' train acc: %f, test acc: %f' 
            %(accuracy_score(y_train, train_preds), accuracy_score(y_test, preds)))
    print()

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Synthetic data
perceptron train acc: 0.994286, test acc: 1.000000
logistic train acc: 0.994286, test acc: 1.000000

Two Moon data
perceptron train acc: 0.820000, test acc: 0.840000
logistic train acc: 0.857143, test acc: 0.866667

Binarized MNIST data
perceptron train acc: 0.873400, test acc: 0.828000
logistic train acc: 0.871000, test acc: 0.834000



# Multiclass classification - SGD vs. GD

Let's get into multiclass classification.
In this question you are going to build a model to classify data into more than just two classes. Also you are going to implement both $SGD$ and $GD$ for multiclass classification and compare performances of the two approaches. Training dataset are similar to question Q2.1, but $ y_{i}\in \left \{ 0,1,...,C-1 \right \} \forall i=1...N.$ Your task is to learn models for multiclass classification based on minimizing logistic loss.

Two of the datasets you are going to do multiclass classification:
* Blur_smile data: <img src="../Blur_smile_data.png" width="400">
* Clear_smile_data data: <img src="../Clear_smile_data.png" width="400">

Here let me give a short review of $SGD$.

From our lecture we know multiclass logistic loss is $$ F\left ( W \right ) = \sum_{n=1}^{N}ln\left ( 1+\sum_{k\neq y_{n}} e^{\left ( w_{k}-w_{y_{n}} \right )^{T}x_{n}}\right ).$$ Now we try to apply $SGD$. First we randomly pick a data $x_{n}$ and minimize logistic loss $$ g\left ( W \right ) = ln\left ( 1+\sum_{k\neq y_{n}} e^{\left ( w_{k}-w_{y_{n}} \right )^{T}x_{n}}\right ).$$ And then find the derivative $\bigtriangledown _{w} g\left ( W \right )$, where $\bigtriangledown _{w} g\left ( W \right )$ is a $CxD$ matrix. 

Let's look at each row k.

If $k\neq y_{n}$: $$\bigtriangledown _{w_{k}} g\left ( W \right )= \frac{e^{\left ( w_{k}-w_{y_{n}} \right )^{T}x_{n}}}{1+\sum_{{k}'\neq y_{n}} e^{\left ( w_{{k}'}-w_{y_{n}} \right )^{T}x_{n}}}x_{n}^{T} = \mathbb{P}\left ( k| x_{n};W\right )x_{n}^{T}$$

else: $$ \bigtriangledown _{w_{k}} g\left ( W \right )= \frac{-\sum_{{k}'\neq y_{n}} e^{\left ( w_{{k}'}-w_{y_{n}} \right )^{T}x_{n}}}{1+\sum_{{k}'\neq y_{n}} e^{\left ( w_{{k}'}-w_{y_{n}} \right )^{T}x_{n}}}x_{n}^{T} = \left (\mathbb{P}\left ( y_{n}| x_{n};W\right )  -1\right )x_{n}^{T}$$

where $\mathbb{P}$ is softmax function.

In the end, our update for $W$ is
$$W \leftarrow W-\eta \begin{bmatrix}\mathbb{P}\left ( y=0 |x_{n};W\right )
\\ \vdots 
\\ \mathbb{P}\left ( y=y_{n} |x_{n};W\right )-1
\\ \vdots 
\\ \mathbb{P}\left ( y=C-1 |x_{n};W\right )
\end{bmatrix} x_{n}^{T}$$

That is the whole idea of $SGD$ of logistic loss for multiclass classification.

* (8 points) ```TODO 6```
Complete the $SGD$ part in ```def multiclass_train```, and don't forget the bias $b$. To randomly pick a data from dataset, you can use ```np.random.choice``` one time in each iteration.

* (16 points) ```TODO 7```
Complete the $GD$ part in ```def multiclass_train```. Compare to $SGD$, $GD$ does not randomly pick a data $x_{n}$. Instead, $GD$ considers all training data points to compute derivative. Please think about how to compute $GD$, and again we want **average** gradient descent. Also there is a tricky point. When dataset is large, $GD$ will takes a large amount of time. How to reduce the time? Make sure you use **numpy programming** instead of nested for loops, otherwise you will not finish your test on Vocareum within the time limit.

Hint 1: If you need to run a for loop for $N$ times to accumulate $C$x$D$ matrices, why not design an equivalent computation as a $C$x$N$ matrix dot another $NxD$ matrix.

Hint 2: You may find it useful to use a special (one-hot) representation of the labels, where each label $y_{i}$ is represented as a row of zeros with a single 1 in the column, that corresponds to the class $y_{i}$. So this one-hot should be an $N$x$C$ matrix.

Advice: To avoid numerical issues such as overflow and underflow caused by ```np.exp```. Let $x$ be a input vector to the softmax function. Use $\tilde{x} = x - max(x)$ instead of using $x$ directly for the softmax function $x$ . That is, if you want to compute $f(x)_{i}$ , compute $f(\tilde{x})_{i}=\frac{exp(\tilde{x}_{i})}{\sum_{j=1}^{D}exp(\tilde{x}_{j})}$ instead, which  is clearly mathematically equivalent but numerically more stable.


* (8 points) ```TODO 8```
You need to complete the predict function in ```def multiclass_predict```. For your convenience, you can use ```np.argmax```.

After you complete ```TODO 6``` ```TODO 7``` ```TODO 8```, please run ```multiclass.sh``` to test your code. If your code is programmed correctly, you should see ```multiclass.out``` as a output file keeping processing time, taining and testing accurancies of $SGD$ and $GD$ for each given dataset. You shall see how fast $SGD$ process compared to $GD$, but its accuracy is hard to catch up with the other's. One more open question to leave you. Is there any chance that $SGD$ reach to the same accuracy as $GD$ does using less time?

In [2]:
from data_loader import smile_dataset_clear, \
                        smile_dataset_blur, \
                        data_loader_mnist 
import time
datasets = [(smile_dataset_clear(), 'Clear smile data', 3) 
            ,(smile_dataset_blur(), 'Blur smile data', 3)
            ,(data_loader_mnist(), 'MNIST', 10)]

for data, name, num_classes in datasets:
    print('%s: %d class classification' % (name, num_classes))
    X_train, X_test, y_train, y_test = data
    for gd_type in ["sgd", "gd"]:
        s = time.time()
        w, b = sol.multiclass_train(X_train, y_train, C=num_classes, gd_type=gd_type)
        print(gd_type + ' training time: %0.6f seconds' %(time.time()-s))
        train_preds = sol.multiclass_predict(X_train, w=w, b=b)
        preds = sol.multiclass_predict(X_test, w=w, b=b)
        print('train acc: %f, test acc: %f' 
            % (accuracy_score(y_train, train_preds), accuracy_score(y_test, preds)))
    print()

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Clear smile data: 3 class classification
sgd training time: 0.108773 seconds
train acc: 0.998358, test acc: 0.996169
gd training time: 0.135419 seconds
train acc: 1.000000, test acc: 1.000000

Blur smile data: 3 class classification
sgd training time: 0.075973 seconds
train acc: 0.937371, test acc: 0.934867
gd training time: 2.246462 seconds
train acc: 0.975743, test acc: 0.974867

MNIST: 10 class classification
sgd training time: 0.103376 seconds
train acc: 0.771800, test acc: 0.773000
gd training time: 14.948777 seconds
train acc: 0.945400, test acc: 0.896000

